# 2020 금융 빅데이터페스티벌 - TEAM 데이터좀아는개미들
## 팀원 : 변영목, 양명한, 이경환

## 1. Making Groups' Characteristics (1차학습 input만들기)

- 4, 5, 6월 데이터를 기준으로 그룹의 특성을 구분하고자 함.
- 각 월별/그룹별 상위 3위의 데이터를 기반으로 우량주선호도, 코스피선호도, 순위변동성,
  통신장비제조업 선호도, 의약품제조업 선호도 및 정보서비스업 선호도를 추출 

In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
# 기본 제공 파일 호출
df = pd.read_csv('./trade_train.csv')
stock = pd.read_csv('./stocks.csv')

df = df.iloc[:, 1:]
stock = stock.iloc[:, 1:]

In [3]:
# 각 월별/그룹별 상위 3개 종목 데이터를 '매수고객수'의 내림차순으로 추출함.
# 2020년 4월 데이터부터 사용.
new_df = df.sort_values(['그룹번호','기준년월','매수고객수'], ascending=[True, True, False])
new_df = new_df.groupby(['그룹번호', '기준년월']).apply(lambda x:x[:3]).iloc[:, 2:].reset_index()
new_df = new_df[['그룹번호', '기준년월', '그룹내고객수', '종목번호', '매수고객수', '매도고객수', '평균매수수량',\
                '매수가격_중앙값', '매도가격_중앙값']]
new_df = new_df[new_df['기준년월']>=202004]
new_df.head()

,그룹번호,기준년월,그룹내고객수,종목번호,매수고객수,매도고객수,평균매수수량,매수가격_중앙값,매도가격_중앙값
27,MAD01,202004,288,A005930,77,49,182,49000,49150
28,MAD01,202004,288,A000660,52,34,107,82657,85000
29,MAD01,202004,288,A005935,28,10,87,42050,42050
30,MAD01,202005,288,A005930,82,46,270,48700,49150
31,MAD01,202005,288,A000660,50,32,193,82500,83407


### 1) 그룹의 우량주 선호도

In [4]:
# 우량주 선정 기준: 전체 기간, 전체 그룹의 매수고객을 합하여 상위 5개 종목
ur = df.groupby('종목번호').매수고객수.sum().sort_values(ascending=False).head(5).reset_index()['종목번호'].values
ur

array(['A005930', 'A000660', 'A035720', 'A005935', 'A005380'],
      dtype=object)

In [5]:
# 월별/그룹별 상위 3개 종목이 우량주에 해당하는지 여부를 컬럼으로 생성.
new_df['우량주여부'] = new_df['종목번호'].isin(ur).astype(int)
new_df.head()

,그룹번호,기준년월,그룹내고객수,종목번호,매수고객수,매도고객수,평균매수수량,매수가격_중앙값,매도가격_중앙값,우량주여부
27,MAD01,202004,288,A005930,77,49,182,49000,49150,1
28,MAD01,202004,288,A000660,52,34,107,82657,85000,1
29,MAD01,202004,288,A005935,28,10,87,42050,42050,1
30,MAD01,202005,288,A005930,82,46,270,48700,49150,1
31,MAD01,202005,288,A000660,50,32,193,82500,83407,1


In [6]:
# 그룹당 월별 상위 3개 종목의 우량주 비율(4,5,6월 총 9개의 상위 종목 중 우량주 개수)
big_love = new_df.groupby('그룹번호').우량주여부.mean().reset_index()
big_love

,그룹번호,우량주여부
0,MAD01,1.000000
1,MAD02,0.666667
2,MAD03,0.111111
3,MAD04,0.666667
4,MAD05,0.444444
5,MAD06,0.666667
6,MAD07,0.444444
7,MAD08,0.000000
8,MAD09,0.555556
9,MAD10,0.444444


### 2) 코스피 선호도 

In [7]:
# 그룹별 코스피 선호도를 측정하기 위해 stock 데이터의 '시장구분' 컬럼을 merge함.
new_df = pd.merge(new_df, stock[['종목번호','시장구분']].drop_duplicates('종목번호'), on='종목번호', how='left')
new_df['시장구분'] = new_df['시장구분'].replace(['코스닥','코스피'],[0,1])

In [8]:
# 그룹의 코스피 선호도(그룹별 9개 상위 종목 중 코스피 종목의 개수)
kospi_love = new_df.groupby('그룹번호').시장구분.mean().reset_index().sort_values('시장구분')
kospi_love

,그룹번호,시장구분
28,MAD29,0.111111
31,MAD32,0.111111
30,MAD31,0.333333
29,MAD30,0.444444
35,MAD36,0.666667
21,MAD22,0.666667
41,MAD42,0.666667
18,MAD19,0.777778
36,MAD37,0.777778
37,MAD38,0.777778


### 3) 그룹별 선호 산업 존재 여부 

- 월별/그룹별 상위 3개 종목 중 특정 산업(표준산업구분코드 중분류 기준)의 비율이 0.5 이상이면, 해당 산업에 대한 선호도가 있다고 가정함.

In [9]:
code = stock[['종목번호','표준산업구분코드_중분류']].drop_duplicates('종목번호')
new_df = pd.merge(new_df, code, on='종목번호', how='left')
area = pd.DataFrame(new_df.groupby('그룹번호').표준산업구분코드_중분류.value_counts()).rename(columns={'표준산업구분코드_중분류':'count'}).reset_index()

# 비율
area['ratio'] = area['count']/9
area

,그룹번호,표준산업구분코드_중분류,count,ratio
0,MAD01,통신장비 제조업,6,0.666667
1,MAD01,전자부품· 컴퓨터· 영상· 음향 및,3,0.333333
2,MAD02,금융업,3,0.333333
3,MAD02,통신장비 제조업,3,0.333333
4,MAD02,전자부품· 컴퓨터· 영상· 음향 및,2,0.222222
...,...,...,...,...
183,MAD48,정보서비스업,3,0.333333
184,MAD48,통신장비 제조업,2,0.222222
185,MAD48,화학물질 및 화학제품 제조업;,2,0.222222
186,MAD48,자동차 및 트레일러 제조업,1,0.111111


In [10]:
# 그룹별로 산업의 비율이 높은 순서대로 정렬해준 후, 제일 높은 비율의 산업만 남김
# 해당 산업의 비율이 0.5 이상인 경우만을 나열함.
area = area.sort_values(['그룹번호','ratio'], ascending=[True, False])
area = area.drop_duplicates('그룹번호', keep='first')
area[area['ratio']>=0.5]['표준산업구분코드_중분류'].unique()

array(['통신장비 제조업', '의료용 물질 및 의약품 제조업', '정보서비스업'], dtype=object)

In [11]:
area['통신장비제조업_선호도'] = (area['표준산업구분코드_중분류'] == '통신장비 제조업') & (area['ratio']>=0.5)
area['의약품제조업_선호도'] = (area['표준산업구분코드_중분류'] == '의료용 물질 및 의약품 제조업') & (area['ratio']>=0.5)
area['정보서비스업_선호도'] = (area['표준산업구분코드_중분류'] == '정보서비스업') & (area['ratio']>=0.5)

area['통신장비제조업_선호도'] = area['통신장비제조업_선호도'].astype(int)
area['의약품제조업_선호도'] = area['의약품제조업_선호도'].astype(int)
area['정보서비스업_선호도'] = area['정보서비스업_선호도'].astype(int)

area

,그룹번호,표준산업구분코드_중분류,count,ratio,통신장비제조업_선호도,의약품제조업_선호도,정보서비스업_선호도
0,MAD01,통신장비 제조업,6,0.666667,1,0,0
2,MAD02,금융업,3,0.333333,0,0,0
6,MAD03,의료용 물질 및 의약품 제조업,8,0.888889,0,1,0
8,MAD04,전자부품· 컴퓨터· 영상· 음향 및,3,0.333333,0,0,0
12,MAD05,정보서비스업,4,0.444444,0,0,0
15,MAD06,정보서비스업,4,0.444444,0,0,0
19,MAD07,코크스· 연탄 및 석유정제품 제조업,4,0.444444,0,0,0
23,MAD08,의료용 물질 및 의약품 제조업,7,0.777778,0,1,0
25,MAD09,정보서비스업,3,0.333333,0,0,0
29,MAD10,통신장비 제조업,4,0.444444,0,0,0


### 4) 순위 변동성
- 그룹 내에서 상위 3위 종목들이 얼마나 자주 바뀌는지를 측정하고자 함.
- 월별로 상위 3위 종목들의 unique한 개수가 많을수록 자주 변동되므로, 해당 값을 사용함.

In [12]:
# 순위변동성
rank = new_df.groupby('그룹번호').종목번호.nunique().reset_index()

In [13]:
# MinMax Scaling
scaler = MinMaxScaler()
rank['종목번호'] = scaler.fit_transform(rank[['종목번호']])

### 5) 통합 : Group Character DataFrame 생성 

In [14]:
# 우량주 선호도
group_character = big_love

In [15]:
# 코스피 선호도
group_character = pd.merge(group_character, kospi_love, on='그룹번호')

In [16]:
# 순위 변동성
group_character = pd.merge(group_character, rank, on='그룹번호')

In [17]:
# 산업 선호도
group_character = pd.merge(group_character, area[['그룹번호', '통신장비제조업_선호도', '의약품제조업_선호도', '정보서비스업_선호도']], on='그룹번호')

In [18]:
# 컬럼명 변경
group_character.rename(columns={'우량주여부':'우량주선호도', '시장구분':'코스피선호도', '종목번호':'순위변동성'}, inplace=True)

In [19]:
group_character

,그룹번호,우량주선호도,코스피선호도,순위변동성,통신장비제조업_선호도,의약품제조업_선호도,정보서비스업_선호도
0,MAD01,1.000000,1.000000,0.0,1,0,0
1,MAD02,0.666667,1.000000,0.4,0,0,0
2,MAD03,0.111111,1.000000,0.6,0,1,0
3,MAD04,0.666667,1.000000,0.2,0,0,0
4,MAD05,0.444444,0.888889,0.4,0,0,0
5,MAD06,0.666667,1.000000,0.4,0,0,0
6,MAD07,0.444444,1.000000,0.4,0,0,0
7,MAD08,0.000000,1.000000,0.4,0,1,0
8,MAD09,0.555556,1.000000,0.6,0,0,0
9,MAD10,0.444444,1.000000,0.2,0,0,0


# 2. 기존의 주어진 데이터를 갖고 활용컬럼 merge하기(2차학습 input만들기)

### stock의 각 월별 거래량과 거래금액을 trade_train.csv에 합쳐주기위한 작업

In [20]:
stock['기준년월'] = stock['기준일자'].astype('str').str[:6].astype('int64')

In [21]:
temp = stock.groupby(['기준년월', '종목번호', '20년7월TOP3대상여부', '시장구분']).agg({'거래량':'sum', '거래금액_만원단위':'sum'}).reset_index()

In [22]:
merged = pd.merge(df, temp, on=['기준년월', '종목번호'], how='left')
merged = merged.dropna(axis=0)

### 이번달의 피쳐들과 다음달의 피쳐들을 한 행에 놓기위한 작업

In [23]:
compare = merged[['기준년월', '그룹번호', '종목번호', '20년7월TOP3대상여부', '시장구분', '평균매수수량','평균매도수량', '매수가격_중앙값', '매도가격_중앙값', '거래량', '거래금액_만원단위', '매수고객수']]

In [24]:
compare = compare.rename({"기준년월": "next_yr"}, axis='columns')

In [25]:
compare['시장구분'] = compare['시장구분'].replace(['코스피', '코스닥'], [1, 0])

In [26]:
next_yr = []
for i in list(merged['기준년월']):
    if i == 202001:
        next_yr.append(201912)
    else:
        next_yr.append(i-1)

In [27]:
merged['next_yr'] = np.array(next_yr)

In [28]:
will_merge = merged[['next_yr', '그룹번호', '종목번호', '매수고객수']]

In [29]:
compare['20년7월TOP3대상여부'] = compare['20년7월TOP3대상여부'].replace(['Y', 'N'], [1, 0])

### 이번달의 피쳐들과 다음달의 피쳐들을 한 행에 놓이게 병합

In [30]:
inputs = pd.merge(compare, will_merge, on=['next_yr', '그룹번호', '종목번호'], how='left')

In [31]:
inputs

,next_yr,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,거래량,거래금액_만원단위,매수고객수_x,매수고객수_y
0,201907,MAD01,A000660,1,1,19,234,74800,78500,68847930.0,516331480.0,7,17.0
1,201907,MAD01,A001820,0,1,533,154,47385,46700,8289994.0,41892101.0,3,NaN
2,201907,MAD01,A004020,1,1,409,528,41411,40000,5896982.0,23727770.0,3,NaN
3,201907,MAD01,A005380,1,1,323,451,140602,140628,11677273.0,157241500.0,4,NaN
4,201907,MAD01,A005930,1,1,34,101,45350,46850,183771791.0,850148952.0,13,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30185,202006,MAD48,A207940,0,1,3,3,835000,797000,7117256.0,537620424.0,3,NaN
30186,202006,MAD48,A272210,1,1,90,42,9900,8870,47504006.0,46461827.0,3,NaN
30187,202006,MAD48,A285130,1,1,271,162,109100,98000,22563031.0,251392053.0,6,NaN
30188,202006,MAD48,A316140,0,1,386,735,10050,9920,54242951.0,51508597.0,5,NaN


# 3. 1과 2의 데이터를 바탕으로 추가 컬럼 만들고 /  1차학습, 2차학습시킨 뒤 예측하기 

In [32]:
# 2에서 만든 데이터
train = inputs[:]
train=train.rename({'next_yr':'기준년월', '매수고객수_x':'이번달_매수고객수', '매수고객수_y':'다음달_매수고객수'}, axis='columns')

In [33]:
# 기간이 4, 5, 6월 데이터만
train = train[train['기준년월'] >= 202004]
train=train[['기준년월', '그룹번호', '종목번호', '20년7월TOP3대상여부', '시장구분', '평균매수수량', '거래량', '거래금액_만원단위', '이번달_매수고객수', '다음달_매수고객수']]

In [34]:
# 우량주여부를 각 종목마다 표현해주는 컬럼만들기! (총 매수 고객 수 상위 5개 종목)
ur = ['A005930', 'A000660', 'A035720', 'A005935', 'A005380']
train['우량주여부'] = train['종목번호'].isin(ur).astype(int)

### 각 월별 매수고객수 3위안에 드는 기업이 바뀌는지 변동성 체크를 위한 컬럼제작

In [35]:
new_df = df.sort_values(['그룹번호','기준년월','매수고객수'], ascending=[True, True, False]).groupby(['그룹번호', '기준년월']).apply(lambda x:x[:3]).iloc[:, 2:].reset_index()
cols = list(new_df.columns)
cols.pop(2)
new_df = new_df[cols]

In [36]:
new_df = new_df[new_df['기준년월']>=202004]

In [37]:
train = pd.merge(train, new_df[['그룹번호', '기준년월', '종목번호', '그룹내_매수여부']], on=['그룹번호', '기준년월', '종목번호'], how='left')

In [38]:
train.rename(columns={'그룹내_매수여부':'이번달_순위권_여부'}, inplace=True)
train['이번달_순위권_여부'].fillna(0, inplace=True)
train['이번달_순위권_여부'] = train['이번달_순위권_여부'].replace('Y', 1) 
train['이번달_순위권_여부'] = train['이번달_순위권_여부'].replace('N', 0) 

### 각 종목번호가 상위 3개의 표준산업구분코드_중분류에 해당하는 지 여부를 나타내기 위한 컬럼 제작

In [39]:
code = stock[['종목번호', '표준산업구분코드_중분류']].drop_duplicates('종목번호')

In [40]:
medic = code[code['표준산업구분코드_중분류'] == '의료용 물질 및 의약품 제조업']['종목번호'].unique()
tongsin = code[code['표준산업구분코드_중분류'] == '통신장비 제조업']['종목번호'].unique()
it = code[code['표준산업구분코드_중분류'] == '정보서비스업']['종목번호'].unique()

In [41]:
train['의약품제조업여부'] = train['종목번호'].isin(medic).astype(int)
train['통신장비제조업여부'] = train['종목번호'].isin(tongsin).astype(int)
train['정보서비스업여부'] = train['종목번호'].isin(it).astype(int)

In [42]:
train

,기준년월,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수,다음달_매수고객수,우량주여부,이번달_순위권_여부,의약품제조업여부,통신장비제조업여부,정보서비스업여부
0,202004,MAD01,A000210,0,1,189,11527212.0,9.136904e+07,5,3.0,0,0,0,0,0
1,202004,MAD01,A000660,1,1,107,91942271.0,7.587119e+08,52,50.0,1,1,0,0,0
2,202004,MAD01,A000990,1,1,544,25807588.0,6.101197e+07,6,6.0,0,0,0,0,0
3,202004,MAD01,A005380,1,1,73,41304577.0,3.862312e+08,12,4.0,1,0,0,0,0
4,202004,MAD01,A005930,1,1,182,424101937.0,2.078519e+09,77,82.0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13145,202006,MAD48,A207940,0,1,3,7117256.0,5.376204e+08,3,NaN,0,0,1,0,0
13146,202006,MAD48,A272210,1,1,90,47504006.0,4.646183e+07,3,NaN,0,0,0,1,0
13147,202006,MAD48,A285130,1,1,271,22563031.0,2.513921e+08,6,NaN,0,0,0,0,0
13148,202006,MAD48,A316140,0,1,386,54242951.0,5.150860e+07,5,NaN,0,0,0,0,0


In [43]:
# 1에서 만들었던 그룹의 특성 데이터
group = group_character[:]

In [44]:
train = pd.merge(train, group, on='그룹번호')

In [45]:
# 매수고객수의 영향력을 조금이나마 줄이기 위한 작업
import numpy as np
train['이번달_매수고객수'] = np.sqrt(train['이번달_매수고객수'])
train['다음달_매수고객수'] = np.sqrt(train['다음달_매수고객수'])

In [46]:
unseen = train[train['기준년월']==202006]

In [47]:
unseen

,기준년월,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수,다음달_매수고객수,우량주여부,이번달_순위권_여부,의약품제조업여부,통신장비제조업여부,정보서비스업여부,우량주선호도,코스피선호도,순위변동성,통신장비제조업_선호도,의약품제조업_선호도,정보서비스업_선호도
65,202006,MAD01,A000660,1,1,121,91590220.0,792877589.0,7.874008,NaN,1,1,0,0,0,1.000000,1.0,0.0,1,0,0
66,202006,MAD01,A000990,1,1,705,23391623.0,68536955.0,2.000000,NaN,0,0,0,0,0,1.000000,1.0,0.0,1,0,0
67,202006,MAD01,A001820,0,1,340,4856906.0,27203237.0,2.000000,NaN,0,0,0,1,0,1.000000,1.0,0.0,1,0,0
68,202006,MAD01,A003620,0,1,3617,342209727.0,121169900.0,1.732051,NaN,0,0,0,0,0,1.000000,1.0,0.0,1,0,0
69,202006,MAD01,A005380,1,1,32,35327068.0,368421899.0,3.000000,NaN,1,0,0,0,0,1.000000,1.0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13145,202006,MAD48,A207940,0,1,3,7117256.0,537620424.0,1.732051,NaN,0,0,1,0,0,0.666667,1.0,0.6,0,0,0
13146,202006,MAD48,A272210,1,1,90,47504006.0,46461827.0,1.732051,NaN,0,0,0,1,0,0.666667,1.0,0.6,0,0,0
13147,202006,MAD48,A285130,1,1,271,22563031.0,251392053.0,2.449490,NaN,0,0,0,0,0,0.666667,1.0,0.6,0,0,0
13148,202006,MAD48,A316140,0,1,386,54242951.0,51508597.0,2.236068,NaN,0,0,0,0,0,0.666667,1.0,0.6,0,0,0


# 1차 학습 데이터, 2차 학습데이터 정의

In [48]:
second_train = train[train['기준년월']<202006][['기준년월', '그룹번호',  '종목번호', '20년7월TOP3대상여부', 
       '시장구분', '평균매수수량', '거래량', '거래금액_만원단위',  '이번달_매수고객수','다음달_매수고객수']]


first_train= train[train['기준년월']<202006][['기준년월', '그룹번호',  '종목번호', '20년7월TOP3대상여부',  '우량주선호도', '코스피선호도',
       '순위변동성', '통신장비제조업_선호도', '의약품제조업_선호도', '정보서비스업_선호도',\
       '시장구분', '우량주여부',
       '이번달_순위권_여부', '의약품제조업여부', '통신장비제조업여부', '정보서비스업여부','다음달_매수고객수']]
first_unseen = unseen[['기준년월', '그룹번호',  '종목번호', '20년7월TOP3대상여부',  '우량주선호도', '코스피선호도',
       '순위변동성', '통신장비제조업_선호도', '의약품제조업_선호도', '정보서비스업_선호도',\
       '시장구분', '우량주여부',
       '이번달_순위권_여부', '의약품제조업여부', '통신장비제조업여부', '정보서비스업여부']]

In [49]:
first_train=first_train.dropna()
second_train=second_train.dropna()

### 1차 학습

In [50]:
from sklearn.ensemble import RandomForestRegressor

rf1 = RandomForestRegressor(n_estimators=150, min_samples_split = 20, random_state=70, min_samples_leaf=8, max_depth= 8, max_leaf_nodes=16,\
                              )\
.fit(first_train.iloc[:, 4:-1].values, first_train.iloc[:, -1].values)

In [51]:
y_pred = rf1.predict(first_unseen.iloc[:, 4:].values)

In [52]:
temp_answer = unseen.copy()
temp_answer['매수고객수'] = y_pred

### 1차 예측 결과를 바탕으로 Re-Rank 시도를 위한 데이터 정리

In [53]:
new_ans = temp_answer[temp_answer['20년7월TOP3대상여부']==1].sort_values(['그룹번호', '매수고객수'], ascending=[True, False])
new_input = new_ans.groupby('그룹번호').apply(lambda x: x[:30]).iloc[:, 2:].reset_index()
new_input = new_input.drop('level_1', axis=1)
new_input = new_input[['그룹번호',  '종목번호', '20년7월TOP3대상여부', 
       '시장구분', '평균매수수량', '거래량', '거래금액_만원단위',  '이번달_매수고객수']]

In [54]:
new_input

,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수
0,MAD01,A005930,1,1,219,517439261.0,2.756149e+09,10.246951
1,MAD01,A005935,1,1,149,59384055.0,2.758364e+08,6.708204
2,MAD01,A000660,1,1,121,91590220.0,7.928776e+08,7.874008
3,MAD01,A035420,1,1,7,26395703.0,6.582895e+08,2.236068
4,MAD01,A005380,1,1,32,35327068.0,3.684219e+08,3.000000
...,...,...,...,...,...,...,...,...
979,MAD48,A105560,1,1,690,64921629.0,2.335226e+08,3.872983
980,MAD48,A272210,1,1,90,47504006.0,4.646183e+07,1.732051
981,MAD48,A285130,1,1,271,22563031.0,2.513921e+08,2.449490
982,MAD48,A096530,1,0,7,39783795.0,4.446157e+08,1.732051


In [55]:
second_train

,기준년월,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수,다음달_매수고객수
0,202004,MAD01,A000210,0,1,189,11527212.0,9.136904e+07,2.236068,1.732051
1,202004,MAD01,A000660,1,1,107,91942271.0,7.587119e+08,7.211103,7.071068
2,202004,MAD01,A000990,1,1,544,25807588.0,6.101197e+07,2.449490,2.449490
3,202004,MAD01,A005380,1,1,73,41304577.0,3.862312e+08,3.464102,2.000000
4,202004,MAD01,A005930,1,1,182,424101937.0,2.078519e+09,8.774964,9.055385
...,...,...,...,...,...,...,...,...,...,...
13099,202005,MAD48,A091990,1,0,143,57006949.0,5.138844e+08,1.732051,2.236068
13101,202005,MAD48,A096530,1,0,8,86460686.0,9.665089e+08,1.732051,1.732051
13102,202005,MAD48,A096770,1,1,17,17847837.0,1.913744e+08,2.000000,2.000000
13103,202005,MAD48,A105560,1,1,343,52428817.0,1.710858e+08,3.000000,3.872983


### 2차 학습

In [56]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(random_state=42, n_estimators=150, max_depth=5, min_child_weight=8, learning_rate=0.008,
                   num_leaves=10)
y_pred = lgb.fit(second_train.iloc[:, 4:-1].values, second_train.iloc[:, -1].values).predict(new_input.iloc[:, 3:].values)
answer = new_input.copy()
answer['매수고객수'] = y_pred

### 순위변동성 및 코스피선호도에 기반한 종목 리랭크 

- 순위변동성이 높은 그룹의 경우, '이번달 매수고객수'가 '다음달 매수고객수'에 큰 영향을 미치지 못할 것이라는 가정.
- 따라서, 2달 전의 거래 종목들을 살펴보고 해당 그룹의 순위를 재조정해주려는 의도.

- 코스피 선호도가 낮은 그룹의 경우, 코스피 종목보다는 코스닥 종목을 많이 매수할 것이라는 가정.
- 따라서, 코스피 선호도가 낮지만 매수고객수 상위 3개 종목에 코스닥 종목이 있다면, 이를 재조정해주고자 함.

- 순위변동성이 높으면서 코스피선호도가 낮은 그룹은 MAD29와 MAD32가 있음.

In [57]:
unseen[['그룹번호', '코스피선호도', '순위변동성']].drop_duplicates('그룹번호').sort_values('코스피선호도')

,그룹번호,코스피선호도,순위변동성
6974,MAD29,0.111111,1.0
7557,MAD32,0.111111,1.0
7290,MAD31,0.333333,0.8
7140,MAD30,0.444444,0.4
8429,MAD36,0.666667,0.0
5391,MAD22,0.666667,0.4
10960,MAD42,0.666667,0.6
4799,MAD19,0.777778,0.2
8713,MAD37,0.777778,0.2
9173,MAD38,0.777778,0.4


In [58]:
# MAD29의 코스피 선호도가 매우 낮음.  -> 0.11
unseen[unseen['그룹번호'] == 'MAD29'].코스피선호도.unique()

array([0.11111111])

In [59]:
# 마찬가지로 MAD32 역시 코스피 선호도가 매우 낮음. 
unseen[unseen['그룹번호'] == 'MAD32'].코스피선호도.unique()

array([0.11111111])

In [60]:
# MAD29 : 저저번달의 거래 종목에서 코스닥인 종목 확인
second_train[(second_train['그룹번호']=='MAD29') & (second_train['기준년월']==202004)\
             & (second_train['20년7월TOP3대상여부']==1) & (second_train['시장구분']==0)].종목번호.unique()

array(['A036540', 'A218410', 'A222080'], dtype=object)

In [61]:
# MAD32 : 저저번달의 거래 종목에서 코스닥인 종목 확인
second_train[(second_train['그룹번호']=='MAD32') & (second_train['기준년월']==202004)\
             & (second_train['20년7월TOP3대상여부']==1) & (second_train['시장구분']==0)].종목번호.unique()

array(['A032850', 'A036540', 'A047820', 'A082270', 'A091990', 'A096530',
       'A133750', 'A215100', 'A218410', 'A222080'], dtype=object)

In [62]:
# 현재 MAD29 그룹에서 상위 3위에 있는 종목들 -> 코스피 종목이 두개. 
# 모두 코스닥 종목으로 리랭크. (A005930, A003000 -> A036540, A218410)
answer[answer['그룹번호'] == 'MAD29'].sort_values('매수고객수', ascending=False)[:3]

,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수,매수고객수
563,MAD29,A222080,1,0,2072,64195811.0,3.376030e+07,3.741657,3.244345
565,MAD29,A005930,1,1,56,517439261.0,2.756149e+09,2.449490,2.699543
564,MAD29,A003000,1,1,283,206174951.0,7.489595e+08,2.000000,2.589025


In [63]:
# 현재 MAD32 그룹에서 상위 3위에 랭크되어 있는 종목들. -> 이미 코스닥 종목으로 채워져있음.
# 따로 리랭크하지 않음.
answer[answer['그룹번호'] == 'MAD32'].sort_values('매수고객수', ascending=False)[:3]

,그룹번호,종목번호,20년7월TOP3대상여부,시장구분,평균매수수량,거래량,거래금액_만원단위,이번달_매수고객수,매수고객수
612,MAD32,A222080,1,0,1970,64195811.0,33760297.0,3.464102,3.215097
613,MAD32,A235980,1,0,147,18270880.0,111029431.0,3.316625,3.209772
601,MAD32,A032850,1,0,114,84749215.0,106286906.0,3.162278,2.909851


### 매수고객수 예측 결과에 따른 정렬 및 제출을 위한 과정

In [64]:
top = stock[stock['20년7월TOP3대상여부'] == 'Y'].종목번호.unique()
stock_name = stock[['종목번호','종목명']].drop_duplicates('종목번호')

group_name = []
for i in range(1, 49):
    group_name.append('MAD{:02d}'.format(i))
    
cols = ['그룹명', '종목번호1', '종목번호2', '종목번호3']

def make_answer(x):
    result = []
    answer = x[x['종목번호'].isin(top)]
    answer = answer.sort_values(by=['그룹번호', '매수고객수'], ascending=[True, False])
    
    answer = answer.groupby('그룹번호').apply(lambda x: x['종목번호'][:3]).reset_index()['종목번호'].values
    
    for i in range(48):
        n = [group_name[i]]
        n.extend(sorted(answer[i*3 : (i+1) * 3]))
        result.append(n)
    return pd.DataFrame(result, columns = cols)

def mapping_name(x):
    x = stock_name[stock_name['종목번호'] == x]['종목명'].values
    return x[0]

In [65]:
result = make_answer(answer)

### 결과 값 (종목명 추출)

In [66]:
result

,그룹명,종목번호1,종목번호2,종목번호3
0,MAD01,A000660,A005930,A005935
1,MAD02,A003490,A005930,A105560
2,MAD03,A003000,A007570,A019170
3,MAD04,A000660,A005930,A068270
4,MAD05,A035420,A035720,A068270
5,MAD06,A005930,A035420,A035720
6,MAD07,A005930,A010950,A096770
7,MAD08,A003000,A007570,A019170
8,MAD09,A005930,A035720,A051910
9,MAD10,A005935,A015760,A030200


In [67]:
# MAD29 그룹에서 기존 코스피 종목을 리랭크.
result.loc[(result['그룹명']=='MAD29'), '종목번호1'] = 'A036540'
result.loc[(result['그룹명']=='MAD29'), '종목번호2'] = 'A218410'

In [68]:
result.iloc[:, 1:].applymap(mapping_name)

,종목번호1,종목번호2,종목번호3
0,SK하이닉스,삼성전자,삼성전자우
1,대한항공,삼성전자,KB금융
2,부광약품,일양약품,신풍제약
3,SK하이닉스,삼성전자,셀트리온
4,NAVER,카카오,셀트리온
5,삼성전자,NAVER,카카오
6,삼성전자,SOil,SK이노베이션
7,부광약품,일양약품,신풍제약
8,삼성전자,카카오,LG화학
9,삼성전자우,한국전력,KT


### csv파일로 저장

In [69]:
result.to_csv('./result.csv', index=False)

In [70]:
result

,그룹명,종목번호1,종목번호2,종목번호3
0,MAD01,A000660,A005930,A005935
1,MAD02,A003490,A005930,A105560
2,MAD03,A003000,A007570,A019170
3,MAD04,A000660,A005930,A068270
4,MAD05,A035420,A035720,A068270
5,MAD06,A005930,A035420,A035720
6,MAD07,A005930,A010950,A096770
7,MAD08,A003000,A007570,A019170
8,MAD09,A005930,A035720,A051910
9,MAD10,A005935,A015760,A030200
